<a href="https://colab.research.google.com/github/thtan8/Coursera_DataScience_Capstone/blob/master/Week3_assignment_Coursera_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading the dataset from Wikipedia. <b>
### Exploring and clustering the neighborhoods in Toronto

In [2]:
import numpy as np
import pandas as pd
import requests
import bs4 as bs
import seaborn as sns
import lxml.html as lh

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Getting the data from the url given.

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(url)
soup = bs.BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
df_data = pd.read_json(df[0].to_json(orient='records'))

In [4]:
df_data.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Filter the dataframe by taking out the "Not Assigned" from the column "Borough" 

In [5]:
df_data_raw = df_data[df_data['Borough'] != 'Not assigned']
df_data_raw.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Grouping the neighborhoods with the same postal code together.

In [6]:
df_data_raw = df_data_raw.groupby(['Postal Code', 'Borough'], as_index=False).agg(','.join)
df_data_raw.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Find in the column "Neighborhood" for "Not assigned" and replace the value by the one in column "Borough".

In [7]:
df_data_raw['Neighbourhood'] = np.where(df_data_raw['Neighbourhood'] == 'Not assigned', df_data_raw['Borough'], df_data_raw['Neighbourhood'])

In [8]:
df_data_raw.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
df_data_raw.shape

(103, 3)

## Getting the latitude and the longitude coordinates of each neighborhood.

From the csv file that has the geographical coordinates of each postal code, to merge with the existing dataframe so that the geospatial coordinates are included in each row.

Retrieving the data from the url given and read into a pandas dataframe.

In [10]:
geospatial_url = "https://cocl.us/Geospatial_data"
geospatial_data = pd.read_csv(geospatial_url)
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


There is a common key "Postal Code" in both the df_data_raw dataframe and the geospatial dataframe. To merge both dataframes together using the postal code as the common key.

In [11]:
df_merged_data = pd.merge(df_data_raw, geospatial_data, on='Postal Code')
df_merged_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
